# What drives the price of a car?

![](images/kurt.jpeg)

**OVERVIEW**

In this application, you will explore a dataset from kaggle. The original dataset contained information on 3 million used cars. The provided dataset contains information on 426K cars to ensure speed of processing.  Your goal is to understand what factors make a car more or less expensive.  As a result of your analysis, you should provide clear recommendations to your client -- a used car dealership -- as to what consumers value in a used car.

### CRISP-DM Framework

<center>
    <img src = images/crisp.png width = 50%/>
</center>


To frame the task, throughout our practical applications we will refer back to a standard process in industry for data projects called CRISP-DM.  This process provides a framework for working through a data problem.  Your first step in this application will be to read through a brief overview of CRISP-DM [here](https://mo-pcco.s3.us-east-1.amazonaws.com/BH-PCMLAI/module_11/readings_starter.zip).  After reading the overview, answer the questions below.

### Business Understanding

From a business perspective, we are tasked with identifying key drivers for used car prices.  In the CRISP-DM overview, we are asked to convert this business framing to a data problem definition.  Using a few sentences, reframe the task as a data task with the appropriate technical vocabulary. 

Perform a regression analysis to model the relationship between the price of used cars (dependent variable) and a variety of features such as make, model, year, mileage, condition, fuel type, and other relevant characteristics (independent variables). The objective is to identify which factors significantly influence the price of used cars, and use this model to make data-driven recommendations to the client, a used car dealership, to optimize their pricing strategy.

##### Key Components:

Target Variable: Used car price (continuous)
Predictor Variables: Vehicle characteristics including but not limited to:

Categorical: make, model, fuel type, transmission, condition
Numerical: year, mileage, engine size, horsepower



##### Methodology:

Conduct exploratory data analysis to identify patterns, distributions, and potential multicollinearity among predictor variables. <br>
Perform feature engineering and selection to optimize model input. <br>
Implement and compare multiple regression techniques (e.g., OLS, Ridge, Lasso, Random Forest) to capture both linear and non-linear relationships. <br>
Evaluate model performance using metrics such as R-squared, RMSE, and MAE. <br>
Analyze feature importance and coefficient estimates to determine the most influential factors on price. <br>
Validate the model using cross-validation techniques to ensure generalizability.

### Data Understanding

After considering the business understanding, we want to get familiar with our data.  Write down some steps that you would take to get to know the dataset and identify any quality issues within.  Take time to get to know the dataset and explore what information it contains and how this could be used to inform your business understanding.

Let me start by loading and inspecting the dataset to identify key features and their relationships to price.  I'll clean the dataset by handling missing values and identify which features are most likely to affect price. <br> It looked like many of these columns have a substantial amount of missing data, so I'll focus on columns with lower missing values and potentially relevant features. I'll proceed by removing rows with missing target values, fill or drop columns with significant missing values, and assess their impact on price. Let's start by cleaning the dataset. 

In [29]:
import pandas as pd
import numpy as np
import datetime
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Load the dataset LOCALLY to inspect it
df = pd.read_csv(r'C:\Users\brean\Documents\BerkAIML\vehicles.csv')

# Display the first few rows of the dataset to understand its structure
df.head(), df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426880 entries, 0 to 426879
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            426880 non-null  int64  
 1   region        426880 non-null  object 
 2   price         426880 non-null  int64  
 3   year          425675 non-null  float64
 4   manufacturer  409234 non-null  object 
 5   model         421603 non-null  object 
 6   condition     252776 non-null  object 
 7   cylinders     249202 non-null  object 
 8   fuel          423867 non-null  object 
 9   odometer      422480 non-null  float64
 10  title_status  418638 non-null  object 
 11  transmission  424324 non-null  object 
 12  VIN           265838 non-null  object 
 13  drive         296313 non-null  object 
 14  size          120519 non-null  object 
 15  type          334022 non-null  object 
 16  paint_color   296677 non-null  object 
 17  state         426880 non-null  object 
dtypes: f

(           id                  region  price  year manufacturer model  \
 0  7222695916                prescott   6000   NaN          NaN   NaN   
 1  7218891961            fayetteville  11900   NaN          NaN   NaN   
 2  7221797935            florida keys  21000   NaN          NaN   NaN   
 3  7222270760  worcester / central MA   1500   NaN          NaN   NaN   
 4  7210384030              greensboro   4900   NaN          NaN   NaN   
 
   condition cylinders fuel  odometer title_status transmission  VIN drive  \
 0       NaN       NaN  NaN       NaN          NaN          NaN  NaN   NaN   
 1       NaN       NaN  NaN       NaN          NaN          NaN  NaN   NaN   
 2       NaN       NaN  NaN       NaN          NaN          NaN  NaN   NaN   
 3       NaN       NaN  NaN       NaN          NaN          NaN  NaN   NaN   
 4       NaN       NaN  NaN       NaN          NaN          NaN  NaN   NaN   
 
   size type paint_color state  
 0  NaN  NaN         NaN    az  
 1  NaN  NaN      

In [31]:
# Check the percentage of missing values in each column
missing_percentage = df.isnull().mean() * 100

# Display columns with missing values sorted by percentage
missing_percentage.sort_values(ascending=False)


size            71.767476
cylinders       41.622470
condition       40.785232
VIN             37.725356
drive           30.586347
paint_color     30.501078
type            21.752717
manufacturer     4.133714
title_status     1.930753
model            1.236179
odometer         1.030735
fuel             0.705819
transmission     0.598763
year             0.282281
id               0.000000
region           0.000000
price            0.000000
state            0.000000
dtype: float64

### Data Preparation

After our initial exploration and fine tuning of the business understanding, it is time to construct our final dataset prior to modeling.  Here, we want to make sure to handle any integrity issues and cleaning, the engineering of new features, any transformations that we believe should happen (scaling, logarithms, normalization, etc.), and general preparation for modeling with `sklearn`. 

In [100]:

# Handle missing values and irrelevant columns
df_cleaned = df.drop(columns=['id', 'region', 'state', 'VIN', 'size']).dropna(subset=['price', 'year', 'odometer'])

# Feature Engineering (car age)
current_year = datetime.datetime.now().year
df_cleaned['car_age'] = current_year - df_cleaned['year']

# Binning the odometer readings
df_cleaned['odometer_bin'] = pd.cut(df_cleaned['odometer'], bins=[0, 50000, 100000, 150000, 200000, 500000],
                                    labels=['Very Low', 'Low', 'Medium', 'High', 'Very High'])

# Limit the number of categories by selecting only the top 10 most common categories for 'model'
top_models = df_cleaned['model'].value_counts().nlargest(10).index
df_cleaned['model'] = df_cleaned['model'].apply(lambda x: x if x in top_models else 'Other')

# Encoding categorical variables including 'odometer_bin'
df_encoded = pd.get_dummies(df_cleaned, columns=['manufacturer', 'fuel', 'transmission', 'condition', 
                                                 'cylinders', 'drive', 'type', 'paint_color', 'title_status', 
                                                 'odometer_bin'])

# Apply label encoding for 'model' to reduce memory usage
label_encoder = LabelEncoder()
df_encoded['model'] = label_encoder.fit_transform(df_cleaned['model'])

# Log transformation for price to handle skewness  
df_encoded['log_price'] = np.log(df_encoded['price'] + 1)

# Scaling numeric features (like odometer and car_age)
scaler = StandardScaler()
df_encoded[['odometer', 'car_age']] = scaler.fit_transform(df_encoded[['odometer', 'car_age']])

# Fill missing values
# 1. Numeric columns: Fill with median
numeric_columns = df_encoded.select_dtypes(include=[np.number]).columns
df_encoded[numeric_columns] = df_encoded[numeric_columns].fillna(df_encoded[numeric_columns].median())

# 2. Categorical columns: Fill with mode
categorical_columns = df_encoded.select_dtypes(exclude=[np.number]).columns
df_encoded[categorical_columns] = df_encoded[categorical_columns].fillna(df_encoded[categorical_columns].mode().iloc[0])

# Splitting the data into features and target variable
X = df_encoded.drop(columns=['price', 'log_price'])  # Exclude price from features
y = df_encoded['log_price']  # Using log-transformed price

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


### Modeling

With your (almost?) final dataset in hand, it is now time to build some models.  Here, you should build a number of different regression models with the price as the target.  In building your models, you should explore different parameters and be sure to cross-validate your findings.

In [ ]:
# Initialize the models
linear_reg = LinearRegression()
random_forest = RandomForestRegressor(random_state=42)

# Cross-validation for Linear Regression
linear_cv_scores = cross_val_score(linear_reg, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
linear_cv_mse = -linear_cv_scores.mean()

# Fit the Linear Regression model
linear_reg.fit(X_train, y_train)
y_pred_linear = linear_reg.predict(X_test)
linear_test_mse = mean_squared_error(y_test, y_pred_linear)

# Cross-validation for Random Forest
rf_cv_scores = cross_val_score(random_forest, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
rf_cv_mse = -rf_cv_scores.mean()

# Fit the Random Forest model
random_forest.fit(X_train, y_train)
y_pred_rf = random_forest.predict(X_test)
rf_test_mse = mean_squared_error(y_test, y_pred_rf)


In [ ]:

# The results
print("Linear Regression CV MSE:", linear_cv_mse)
print("Linear Regression Test MSE:", linear_test_mse)
print("\nRandom Forest CV MSE:", rf_cv_mse)
print("Random Forest Test MSE:", rf_test_mse)

### Evaluation

With some modeling accomplished, we aim to reflect on what we identify as a high quality model and what we are able to learn from this.  We should review our business objective and explore how well we can provide meaningful insight on drivers of used car prices.  Your goal now is to distill your findings and determine whether the earlier phases need revisitation and adjustment or if you have information of value to bring back to your client.

Our models, particularly the random forest, reveal key drivers of used car prices like car age, odometer reading, and manufacturer. However, the MSE indicates room for improvement, suggesting that non-linear relationships are important. While we offer valuable insights on significant price determinants, such as the impact of specific manufacturers and vehicle conditions, the models’ accuracy may not fully meet our business objective.

To enhance predictions, revisiting earlier phases, especially in feature engineering and data cleaning, is advisable. Incorporating additional data or refining categorical encodings could strengthen our models, ultimately providing more actionable insights for the client.

### Deployment

Now that we've settled on our models and findings, it is time to deliver the information to the client.  You should organize your work as a basic report that details your primary findings.  Keep in mind that your audience is a group of used car dealers interested in fine tuning their inventory.

##### 1. Introduction
This report presents an analysis of the key factors influencing used car prices, aimed at helping used car dealers optimize their inventory pricing strategies. We employed linear regression and random forest models to identify the primary drivers of car prices and provide actionable insights.

##### 2. Data Overview
We analyzed a dataset containing various attributes of used cars, including car age, odometer readings, manufacturer, model, transmission type, and condition. The dataset underwent rigorous cleaning and feature engineering to ensure the accuracy and relevance of our findings.

##### 3. Key Findings
Primary Price Drivers:

Car Age: Older vehicles generally command lower prices, with depreciation accelerating significantly after the first few years.<br>
Odometer Reading: Higher mileage is strongly associated with lower prices, reflecting the wear and tear on the vehicle.<br>
Manufacturer: Certain manufacturers, such as Toyota and Honda, consistently command higher prices due to brand reputation for reliability and durability.<br>
Condition and Transmission: Cars in better condition and with automatic transmissions tend to sell at higher prices. <br>
Model Performance:<br>

The random forest model outperformed the linear regression model, indicating that non-linear relationships and interactions between variables are crucial in accurately predicting car prices. <br>

##### 4. Business Insights
The analysis confirms that car age, mileage, and brand are critical in determining prices. Dealers should focus on acquiring and promoting vehicles with lower mileage, from reputable manufacturers, and in good condition to maximize profits.

##### 5. Recommendations
Inventory Selection: Prioritize vehicles with lower mileage and from manufacturers known for reliability.
Pricing Strategy: Adjust prices based on vehicle age and condition, leveraging insights from the model to fine-tune pricing more precisely.
Further Analysis: Consider incorporating additional data, such as market trends or regional demand, to further enhance pricing strategies.
##### 6. Conclusion
Our findings provide valuable insights that can help used car dealers optimize their inventory pricing. By focusing on the identified key drivers, dealers can improve their profitability and better meet market demands. Further refinement of the models, including additional data and more advanced techniques, could offer even greater precision in pricing strategies.